# Introducción

En esta segunda práctica diseñaremos las primeras etapas para llegar a construir **funciones de ranking para ordenar los resultados de un motor de búsqueda**. Utilizaremos un _dataset_ de entrenamiento que incluye numerosas consultas, cada una con un conjunto limitado de resultados (habitualmente unos 10 por consulta). Este dataset se basa en un corpus de ~100K documentos y ~340K términos, extraídos de un motor de búsqueda comercial aplicado a la web de la Universidad de Stanford, por lo que es bastante realista. El corpus ha sido preprocesado para optimizar las necesidades de almacenamiento y cómputo del notebook. Todas estas características lo hacen idóneo para las prácticas de la asignatura.

Para cada par consulta(_q_)-documento(_d_), se proporcionan varias características (_features_) que serán de utilidad para realizar este _ranking_. Se proporciona también un conjunto de entrenamiento con pares consulta-documentos en los que a cada documento se le ha asignado, manualmente, un valor de relevancia (etiquetado previamente), que será utilizado fundamentalmente para medir a posteriori la calidad de las funciones de ranking implementadas.

## Imports necesarios

In [1]:
import sys
import array
import os
import timeit
import contextlib
import math
import urllib.request
import zipfile
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from collections import OrderedDict, Counter, defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
!mkdir -p output # Creación de subdirectorio auxiliar para salidas

# Dataset

Los datos para esta práctica están disponibles como archivo .zip en este [enlace](http://web.stanford.edu/class/cs276/pa/pa3-data.zip). El _dataset_ está dividido en dos conjuntos separados:
1. **Conjunto de entrenamiento** formado por 731 consultas (`pa3.(signal|rel).train`)
2. **Conjunto de test** formado por 124 consultas (`pa3.(signal|rel).dev`)

La idea será que, a la vez que ajustemos y maximicemos el rendimiento en el conjunto de entrenamiento, verifiquemos el rendimiento de los parámetros ajustados en el conjunto de desarrollo para asegurarnos de que no estamos sobreajustando el modelo.

In [3]:
# Descarga del dataset:
data_dir = 'pa3-data'
data_url = 'http://web.stanford.edu/class/cs276/pa/{}.zip'.format(data_dir)
urllib.request.urlretrieve(data_url, '{}.zip'.format(data_dir))

# Descomprimimos el archivo .zip:
with zipfile.ZipFile('{}.zip'.format(data_dir), 'r') as zip_fh:
    zip_fh.extractall()
print('Datos descargados y descomprimidos en {}...\n'.format(data_dir))

# Imprimimos la estructura del directorio:
print('Estructura del directorio:')
print(data_dir + os.path.sep)
for sub_dir in os.listdir(data_dir):
    if not sub_dir.startswith('.'):
        print('  - ' + sub_dir)

Datos descargados y descomprimidos en pa3-data...

Estructura del directorio:
pa3-data/
  - pa3.rel.train
  - pa3.signal.dev
  - pa3.signal.train
  - pa3.rel.dev
  - docs.dict
  - terms.dict
  - BSBI.dict


## Descripción de los archivos

### Archivos de "señal"

- **pa3.signal.(_train_|_test_)**: Contienen el conjunto de consultas correspondiente (_train_|_test_) junto con los documentos devueltos para cada consulta individual por un determinado motor de búsqueda. La lista de documentos se encuentra convenientemente mezclada y no está en el mismo orden que el devuelto por el motor de búsqueda empleado originalmente. Cada consulta tiene siempre 10 o menos documentos de respuesta. Por ejemplo, el formato de un par consulta-documento $(q,d)$ es el siguiente:

In [4]:
# Mostramos un conjunto de líneas ilustrativo del archivo de training (la primera
# consulta ("query:"), con sus primeros dos documentos ("url:") completos (líneas 0 a 29),
# y otro par consulta-documentos adicional más adelante en el fichero (líneas 233
# en adelante):
filename = os.path.join(data_dir, "pa3.signal.train")
with open(filename, 'r', encoding = 'utf8') as f:
    lines = f.readlines()
    for l in lines[0:29]:
        print(l, end="")
    print("    ...\n")
    for l in lines[233:256]:
        print(l, end="")
    print("    ...")

query: stanford aoerc pool hours
  url: http://events.stanford.edu/2014/February/18/
    title: events at stanford tuesday february 18 2014
    header: stanford university event calendar
    header: teaching sex at stanford
    header: rodin the complete stanford collection
    header: stanford rec trx suspension training
    header: memorial church open visiting hours
    header: alternative transportation counseling tm 3 hour stanford univ shc employees retirees family members
    body_hits: stanford 239 271 318 457 615 642 663 960 966 971
    body_hits: aoerc 349 401 432 530 549 578 596
    body_hits: pool 521
    body_length: 981
    pagerank: 1
  url: http://events.stanford.edu/2014/February/6/
    title: events at stanford thursday february 6 2014
    header: stanford university event calendar
    header: stanford woods environmental forum featuring roz naylor
    header: stanford school of earth sciences alumni reception at nape
    header: an evening with stanford alumnus and p

Atendiendo a la salida anterior se ejemplifica perfectamente la estructura de estos archivos de señal. El patrón se repite para cada URL, hasta que todas las URL (siempre 10 o menos) de la correspondiente consulta estén completas. A continuación este patrón general se repite para cada consulta posterior. Hay solo un campo `title`, otro `pagerank`, y otro `body_length` para cada URL, pero puede haber múltiples campos `header`, `body_hits` y `anchor_text` (junto con los correspondientes `stanford_anchor_count`). He aquí el significado de los mencionados campos:

* El campo `title` es el título de la página.

* El campo `pagerank` es un número entero de 0 a 9 que indica una calidad independiente de la consulta de la página (cuanto mayor sea dicho valor, mejor será la calidad de la página).

* El campo `body_length` indica cuántos términos están presentes en el cuerpo del documento.

* Cada campo `header` indica un subtítulo contenido en la página.

* Cada campo `body_hits` especifica, para cada término de la consulta $q$, la _posting list_ posicional para ese término en el documento (posiciones de la palabra en el mismo, siempre ordenadas en orden creciente).

* Cada campo `anchor_text`, siempre seguido inmediatamente por un subcampo `stanford_anchor_count` correspondiente, indica el texto de un enlace desde cualquier página del dominio de páginas web en las que está basado el dataset ([https://www.stanford.edu/](https://www.stanford.edu/)) al documento actual, junto con el número total de enlaces con dicho texto en el dataset. Por ejemplo, si el texto de anclaje (`anchor_text`) es _"Stanford math department"_ y el recuento (`stanford_anchor_count`) es 9, eso significa que hay nueve enlaces a la página actual (desde otras páginas en https://www.stanford.edu/) donde el texto de anclaje es exactamente _"Stanford math department"_. Así, en el ejemplo anterior del segundo documento de la segunda consulta, podemos ver que el anclaje _"Cardinal nights"_ aparece en 208 páginas del dominio https://www.stanford.edu/ apuntando al documento https://alcohol.stanford.edu/cardinal-nights en cuestión.

El campo `pagerank` será empleado más adelante como ejemplo de _feature_ no textual, mientras que el resto de campos se corresponden con distintas _zonas_ del documento, que actuarán por tanto como _features_ textuales diferenciadas.


### Archivos de relevancias

* **pa3.rel.(train|dev)**: Estos archivos contienen una lista de juicios de relevancia (etiquetados manualmente) para cada par consulta-documento $(q,d)$ en los respectivos archivos de señal (_train|test_). El valor de relevancia es siempre un entero entre -1 y 3, con un dato mayor significando que el documento tiene más relevancia. −1 significaría que el documento ha sido simplemente ignorado. De nuevo, el patrón se repite para cada consulta, hasta que se alcanza el final del archivo.

Por ejemplo:

In [5]:
# Mostramos las primeras filas del archivo:
filename = os.path.join(data_dir, "pa3.rel.train")
with open(filename, 'r', encoding = 'utf8') as f:
    lines = f.readlines()
    for l in lines[0:25]:
        print(l.strip())
print("...")

query: stanford aoerc pool hours
url: http://events.stanford.edu/2014/February/18/ 0.0
url: http://events.stanford.edu/2014/February/6/ 0.0
url: http://events.stanford.edu/2014/March/13/ 0.0
url: http://events.stanford.edu/2014/March/3/ 0.0
url: http://med.stanford.edu/content/dam/sm/hip/documents/FreeFitnessWeek.pdf 0.0
url: http://web.stanford.edu/group/masters/pool.html 1.0
url: https://alumni.stanford.edu/get/page/perks/PoolAndGyms 1.5
url: https://cardinalrec.stanford.edu/facilities/aoerc/ 2.0
url: https://explorecourses.stanford.edu/search?view=catalog&filter-coursestatus-Active=on&page=0&catalog=&q=PE+128%3A+Swimming%3A+Beginning+I&collapse= 0.5
url: https://glo.stanford.edu/events/stanford-rec-open-house 0.5
query: alumni association benefits
url: http://alumni.stanford.edu/get/page/membership/benefits/creditcard 2.0
url: http://alumni.stanford.edu/get/page/membership/benefits/libraries 2.0
url: http://alumni.stanford.edu/get/page/membership/students 2.0
url: https://alumni-esc

Finalmente, las funciones de _ranking_ requieren también ciertos estadísticos a nivel de colección de documento (tales como la frecuencia inversa de documento, o _idf_), que no están contenidos en los anteriores archivos. Para eso proporcionamos los archivos **docs.dict**, **terms.dict** y **BSBI.dict**, con los que pueden calcularse los correspondientes valores de _idf_ necesarios. En este caso, se trata simplemente de archivos binarios (`pickle` de python):

In [6]:
! file pa3-data/*.dict

pa3-data/BSBI.dict:  data
pa3-data/docs.dict:  data
pa3-data/terms.dict: data


## _Parsing_ de los archivos del _dataset_

En primer lugar, definimos las clases `Query` y `Document` y la función `load_train_data`, necesarias para hacer el _parsing_ de los datos textuales contenidos en los ficheros de entrada:

### Clase _Query_

In [7]:
class Query:
    """Clase utilizada para almacenar una consulta."""
    def __init__(self, query):
        self.query_words = query.split(" ")

    def __iter__(self):
        for w in self.query_words:
            yield w

    def __eq__(self, other):
        if not isinstance(other, Query):
            return False
        return self.query_words == other.query_words

    def __hash__(self):
        return hash(str(self))

    def __str__(self):
        return " ".join(self.query_words)

    __repr__ = __str__

### Clase _Document_

In [8]:
class Document:
    """Clase utilizada para almacenar información útil para un documento."""
    def __init__(self, url):
        self.url = url        # Cadena
        self.title = None     # Cadena
        self.headers = None   # [Lista de cadenas]
        self.body_hits = None # Diccionario: Término->[Lista de posiciones]
        self.body_length = 0  # Entero
        self.pagerank = 0     # Entero
        self.anchors = None   # Diccionario: Cadena->[Conteo total de ocurrencias (anchor_counts)]

    def __iter__(self):
        for u in self.url:
            yield u

    def __str__(self):
        result = [];
        NEW_LINE = "\n"
        result.append("url: "+ self.url + NEW_LINE);
        if (self.title is not None): result.append("title: " + self.title + NEW_LINE);
        if (self.headers is not None): result.append("headers: " + str(self.headers) + NEW_LINE);
        if (self.body_hits is not None): result.append("body_hits: " + str(self.body_hits) + NEW_LINE);
        if (self.body_length != 0): result.append("body_length: " + str(self.body_length) + NEW_LINE);
        if (self.pagerank != 0): result.append("pagerank: " + str(self.pagerank) + NEW_LINE);
        if (self.anchors is not None): result.append("anchors: " + str(self.anchors) + NEW_LINE);
        return " ".join(result)

    __repr__ = __str__

### Función _load_train_data_

La siguiente función realiza el _parsing_ de toda la información textual relevante contenida en un archivo de _training | test_ dado, ya en una estructura python (`query_dict`) fácilmente accesible programáticamente:

In [9]:
def load_train_data(feature_file_name):
    """
    Args:
        feature_file_name: Camino al fichero con las features de entrada.

    Returns:
       query_dict: Diccionario de tipo "Consulta -> (URL -> Documento)"". Por ejemplo:
        {computer science master: {'http://cs.stanford.edu/people/eroberts/mscsed/Admissions-MSInCSEducation.html':
          {title: ms in computer science education stanford computer science
           headers: ["master's degree in computer science education"]
           body_hits: {'computer': [15], 'science': [16]}
           body_length: 741
           anchors: {'computer science': 2},
          'http://scpd.stanford.edu/online-engineering-courses.jsp': title: online engineering courses stanford university
           headers: ['computer science and information technology']
           body_hits: {'science': [136], 'master': [188], 'computer': [223]}
           body_length: 687,
           }
         ...,
         }
    """
    line = None
    url = None
    anchor_text = None
    query = None
    query_dict = {}
    try:
        with open(feature_file_name, 'r', encoding = 'utf8') as f:
            for line in f:
                token_index = line.index(":")
                key = line[:token_index].strip()
                value = line[token_index + 1:].strip()
                if key == "query":
                    # Nueva consulta:
                    query = Query(value)
                    query_dict[query] = {}
                elif key == "url":
                    # Nuevo documento encontrado para la actual consulta:
                    url = value;
                    query_dict[query][url] = Document(url);
                elif key == "title":
                    query_dict[query][url].title = str(value);
                elif key == "header":
                    if query_dict[query][url].headers is None:
                        query_dict[query][url].headers = []
                    query_dict[query][url].headers.append(value)
                elif key == "body_hits":
                    if query_dict[query][url].body_hits is None:
                        query_dict[query][url].body_hits = {}
                    temp = value.split(" ",maxsplit=1);
                    term = temp[0].strip();
                    if term not in query_dict[query][url].body_hits:
                        positions_int = []
                        query_dict[query][url].body_hits[term] = positions_int
                    else:
                        positions_int = query_dict[query][url].body_hits[term]
                    positions = temp[1].strip().split(" ")
                    for position in positions:
                        positions_int.append(int(position))
                elif key == "body_length":
                    query_dict[query][url].body_length = int(value);
                elif key == "pagerank":
                    query_dict[query][url].pagerank = int(value);
                elif key == "anchor_text":
                    anchor_text = value
                    if query_dict[query][url].anchors is None:
                        query_dict[query][url].anchors = {}
                elif key == "stanford_anchor_count":
                    query_dict[query][url].anchors[anchor_text] = int(value)
    except FileNotFoundError:
        print(f"Fichero {feature_file_name} no encontrado!")

    return query_dict

# Cargamos archivo completo de training:
file_name = os.path.join(data_dir, "pa3.signal.train")
query_dict = load_train_data(file_name)

Probamos ahora el mapeo generado (`query_dict`) para unas cuantas entradas del fichero `"pa3.signal.train"`. Obsérvese que éste es de tipo diccionario python _Consulta->(URL->Documento)_, y por tanto posibles usos válidos de este mapeo serían, por ejemplo, los siguientes:

* Acceso a todos los documentos resultado de una consulta:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`query_dict[Query("stanford aoerc pool hours")]`

* Acceso a un documento:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`query_dict[Query("stanford aoerc pool hours")]['http://events.stanford.edu/2014/February/18/']`

* Acceso a un determinado campo de un documento:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`query_dict[Query("stanford aoerc pool hours")]['http://events.stanford.edu/2014/February/18/'].body_hits`

In [10]:
# Acceder a todos los documentos (dados por sus URLs) de una consulta:
query_text = "stanford aoerc pool hours"
query = query_dict[Query(query_text)]
print(f'Documentos (URLs) para la consulta "{query_text}":')
for url in query.keys():
    print(f"  {url}")

# Acceder a un documento completo, dado por su URL, dentro de la consulta:
query_text = "stanford aoerc pool hours"
url_text = "http://events.stanford.edu/2014/February/18/"
print(f'\nDocumento "{url_text}" correspondiente a la consulta "{query_text}":')
print(query_dict[Query(query_text)][url_text])

# Acceder sólo a un campo específico dentro de un documento:
print(f'Campo "body_hits" para documento "{url_text} correspondiente a la consulta "{query_text}:"')
print(query_dict[Query(query_text)][url_text].body_hits)

# Acceder a otros campos de otro documento, correspondiente a otra consulta diferente:
query_text = "alumni association benefits"
url_text = "http://alumni.stanford.edu/get/page/membership/benefits/creditcard"
sample_doc = query_dict[Query(query_text)][url_text]
print(f"\nVarios campos individuales del par consulta-documento {query_text}-{url_text}:\b")
print("  url:", sample_doc.url)
print("  headers:", sample_doc.headers)
print("  body_hits:",sample_doc.body_hits)

Documentos (URLs) para la consulta "stanford aoerc pool hours":
  http://events.stanford.edu/2014/February/18/
  http://events.stanford.edu/2014/February/6/
  http://events.stanford.edu/2014/March/13/
  http://events.stanford.edu/2014/March/3/
  http://med.stanford.edu/content/dam/sm/hip/documents/FreeFitnessWeek.pdf
  http://web.stanford.edu/group/masters/pool.html
  https://alumni.stanford.edu/get/page/perks/PoolAndGyms
  https://cardinalrec.stanford.edu/facilities/aoerc/
  https://explorecourses.stanford.edu/search?view=catalog&filter-coursestatus-Active=on&page=0&catalog=&q=PE+128%3A+Swimming%3A+Beginning+I&collapse=
  https://glo.stanford.edu/events/stanford-rec-open-house

Documento "http://events.stanford.edu/2014/February/18/" correspondiente a la consulta "stanford aoerc pool hours":
url: http://events.stanford.edu/2014/February/18/
 title: events at stanford tuesday february 18 2014
 headers: ['stanford university event calendar', 'teaching sex at stanford', 'rodin the comple

## Construcción del diccionario IDF

Construiremos ahora la clase que nos permitirá computar los IDF de los términos a partir de los correspondientes archivos <b>.dict</b> que usaremos también como entrada (similares a los utilizados en el _notebook_ de la práctica 1).



### Clase _IdMap_

Comenzamos creando la clase IdMap, para mapear cadenas (_tokens_) a sus correspondientes identificadores numéricos (_tokenIDs_) y viceversa. Valdrá tanto para documentos (identificados por su URL) como para términos (cadenas con un simple token):


In [11]:
class IdMap:
    """Clase auxiliar para almacenar mapeos entre strings e identificadores numéricos de tokens."""
    
    def __init__(self):
        """Constructor de la clase IdMap."""
        self.str_to_id = {}  # Diccionario: string -> id numérico
        self.id_to_str = []  # Lista: índice = id, valor = string
        
    def __len__(self):
        """Devuelve el número de elementos en el mapeo."""
        return len(self.id_to_str)
    
    def _get_str(self, i):
        """Devuelve el string correspondiente al id numérico i.
        
        Args:
            i (int): Identificador numérico.
            
        Returns:
            str: String correspondiente al id.
        """
        return self.id_to_str[i]
    
    def _get_id(self, s):
        """Devuelve el id numérico correspondiente al string s.
        Si el string no existe, lo añade y devuelve su nuevo id.
        
        Args:
            s (str): String del que obtener su id.
            
        Returns:
            int: Identificador numérico del string.
        """
        if s not in self.str_to_id:
            # Si no existe, lo añadimos
            new_id = len(self.id_to_str)
            self.str_to_id[s] = new_id
            self.id_to_str.append(s)
            return new_id
        return self.str_to_id[s]
    
    def __getitem__(self, key):
        """Permite acceso mediante corchetes: idmap[key]
        Si key es int, devuelve el string.
        Si key es str, devuelve el id.
        
        Args:
            key: Puede ser int (devuelve string) o str (devuelve id).
            
        Returns:
            El string o id correspondiente.
        """
        if isinstance(key, int):
            return self._get_str(key)
        elif isinstance(key, str):
            return self._get_id(key)
        else:
            raise TypeError("La clave debe ser int o str")
    
    def __contains__(self, key):
        """Permite usar 'in' para verificar existencia.
        
        Args:
            key: Puede ser int o str.
            
        Returns:
            bool: True si existe, False en caso contrario.
        """
        if isinstance(key, int):
            return 0 <= key < len(self.id_to_str)
        elif isinstance(key, str):
            return key in self.str_to_id
        return False
    
    def __repr__(self):
        """Representación en string del objeto."""
        return f"IdMap(size={len(self)})"

### Lectura de diccionarios (_docs_, _terms_, _postings\_dict_)

Cargamos ahora los archivos `docs.dict`, `terms.dict` y `BSBI.dict` creados en la práctica 1, e imprimimos sus respectivas longitudes y los primeros registros para recordar sus respectivas estructuras:

In [12]:
# Leemos los tres archivos e imprimimos sus longitudes:
with open("pa3-data/terms.dict", 'rb') as f:
    terms = pkl.load(f)
with open("pa3-data/docs.dict", 'rb') as f:
    docs = pkl.load(f)
with open('pa3-data/BSBI.dict', 'rb') as f:
    postings_dict, termsID = pkl.load(f)
print(f"Leídos {len(docs)} documentos y {len(terms)} términos")

# Chequeamos la consistencia de longitudes:
assert len(terms) == len(termsID) == len(postings_dict), \
       "Inconsistencia en longitudes de terms, termsID y/o postings_dict"

# Imprimimos los primeros diez valores de los idMaps docs y terms, y también de postings_dict y termsId:
print("----------- terms: -----------")
print("10 primeros términos:         ", terms.id_to_str[:10])
print("10 primeros mappings term->id:", list(zip(list(terms.str_to_id.keys()),terms.str_to_id.values()))[:10])
print("\n--------- docs: ------------")
print("10 primeros documentos:       ", docs.id_to_str[:10])
print("10 primeros mappings doc->id: ", list(zip(list(docs.str_to_id.keys()),docs.str_to_id.values()))[:10])
print("\n------ postings_dict: ------")
print("10 primeros postings_dict:    ", list(zip(list(postings_dict.keys()),postings_dict.values()))[:10])
print("10 primeros termsID:          ", termsID[:10])

Leídos 98998 documentos y 347071 términos
----------- terms: -----------
10 primeros términos:          ['3d', 'radiology', 'lab', 'stanford', 'university', 'school', 'of', 'medicine', 'and', 'quantitative']
10 primeros mappings term->id: [('3d', 0), ('radiology', 1), ('lab', 2), ('stanford', 3), ('university', 4), ('school', 5), ('of', 6), ('medicine', 7), ('and', 8), ('quantitative', 9)]

--------- docs: ------------
10 primeros documentos:        ['0/3dradiology.stanford.edu_', '0/3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html', '0/3dradiology.stanford.edu_patient_care_case_studies.html', '0/5-sure.stanford.edu_', '0/50years.stanford.edu_', '0/a3cservices.stanford.edu_awards_nominate_', '0/a3cservices.stanford.edu_facilities_', '0/a3cservices.stanford.edu_lead_', '0/aa.stanford.edu_', '0/aa.stanford.edu_about_aviation.php']
10 primeros mappings doc->id:  [('0/3dradiology.stanford.edu_', 0), ('0/3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html', 1), ('

### Clase _Idf_

A continuación, la clase `Idf` que, a partir de los diccionarios anteriores, calcula la frecuencia (absoluta) de cada término, y el correspondiente valor IDF pesado logarítmicamente, como es habitual:

In [13]:
class Idf:
    """Construye un diccionario para poder devolver el IDF de un término (tanto si el término consultado está
       como si no está en el diccionario construido).
       Recuérdese de la práctica 1 que el diccionario "postings_dict" mapea cada termID a una tupla
       (posicion_de_comienzo_en_fichero_indice, numero_de_postings_en_la_lista, longitud_en_bytes_de_la_lista).
       Dado que queremos protegernos del caso posible de consulta de un término que no aparezca en la colección,
       aplicaremos el suavizado de Laplace típico (suma de +1 una unidad tanto en el numerador como en el
       denominador de la proporción, evitando así la posible división por cero).
    """
    def __init__(self):
        """Construcción del diccionario IDF"""
        try:
            with open("pa3-data/docs.dict", 'rb') as f:
                docs = pkl.load(f)
            self.total_doc_num = len(docs)
            print("Número total de documentos de la colección:", self.total_doc_num)

            with open("pa3-data/terms.dict", 'rb') as f:
                terms = pkl.load(f)
            self.total_term_num = len(terms)
            print("Número total de términos:", self.total_term_num)

            with open('pa3-data/BSBI.dict', 'rb') as f:
                postings_dict, termsID = pkl.load(f)

            self.idf = {}
            self.raw = {}
            ### BEGIN YOUR CODE (FIXME)
            for term_id, term_str in enumerate(terms.id_to_str):
                # El número de postings es el segundo elemento de la tupla en postings_dict
                # postings_dict[term_id] = (posicion, num_postings, longitud_bytes)
                if term_id in postings_dict:
                    num_docs_with_term = postings_dict[term_id][1]  # número de documentos que contienen el término
                    self.raw[term_str] = num_docs_with_term
                    # Aplicar suavizado de Laplace y calcular IDF logarítmico
                    # IDF = log((N + 1) / (df + 1))
                    self.idf[term_str] = math.log((self.total_doc_num + 1) / (num_docs_with_term + 1))
            ### END YOUR CODE (FIXME)
        except FileNotFoundError:
            print("¡Ficheros de diccionario de documentos / términos / índice no encontrados!")

    def get_raw(self, term):
        """Devuelve el conteo crudo de ocurrencias de documentos conteniendo un término,
           dado, tanto si está como si no en el diccionario.
        Args:
            term(str) : Término del que se va a devolver su conteo crudo.
        Return(float):
            Idf del término.
        """
        ### BEGIN YOUR CODE (FIXME)
        # Si el término existe en el diccionario, devolver su conteo
        # Si no existe, devolver 0
        return self.raw.get(term, 0)
        ### END YOUR CODE (FIXME)

    def get_idf(self, term):
        """Devuelve el IDF de un término, tanto si está como si no en el diccionario.
        Args:
            term(str) : Término del que se va a devolver su IDF.
        Return(float):
            Idf del término.
        """
        ### BEGIN YOUR CODE (FIXME)
        # Si el término existe en el diccionario, devolver su IDF
        # Si no existe, aplicar suavizado de Laplace con df=0
        if term in self.idf:
            return self.idf[term]
        else: 
            return math.log(self.total_doc_num + 1)
        ### END YOUR CODE (FIXME)

Creamos ahora una instancia de esta clase `Idf`, y probamos a consultar en ella la IDF de unos cuantos términos, tanto existentes como inexistentes. Obsérvese que, a mayor frecuencia (términos más comunes en el corpus) de un término, menor peso IDF, y viceversa (términos menos comunes tienen un mayor peso IDF). Del mismo modo, un término inexistente tendrá un IDF muy cercano a 5.0 (ya que la cantidad total de documentos es aproximadamente 100K=$10^5$):

In [14]:
theIDF = Idf()
print()
print(f'IDF("the") =                  {theIDF.get_idf("the"):5.3f}  (raw count={theIDF.get_raw("the")})')
print(f'IDF("and") =                  {theIDF.get_idf("and"):5.3f}  (raw count={theIDF.get_raw("and")})')
print(f'IDF("stanford") =             {theIDF.get_idf("stanford"):5.3f}  (raw count={theIDF.get_raw("stanford")})')
print(f'IDF("university") =           {theIDF.get_idf("university"):5.3f}  (raw count={theIDF.get_raw("university")})')
print(f'IDF("quantitative") =         {theIDF.get_idf("quantitative"):5.3f}  (raw count={theIDF.get_raw("quantitative")})')
print(f'IDF("supercalifragilistic") = {theIDF.get_idf("supercalifragilistic"):5.3f}  (raw count={theIDF.get_raw("supercalifragilistic")})')

Número total de documentos de la colección: 98998
Número total de términos: 347071

IDF("the") =                  0.191  (raw count=81770)
IDF("and") =                  0.217  (raw count=79688)
IDF("stanford") =             0.330  (raw count=71202)
IDF("university") =           0.557  (raw count=56709)
IDF("quantitative") =         5.164  (raw count=565)
IDF("supercalifragilistic") = 11.503  (raw count=0)


Lo siguiente son unos cuantos tests adicionales, que aseguran la corrección de la implementación de la clase:

In [15]:
assert len(theIDF.idf) == 347071, 'Longitud incorrecta de diccionario idf.'
assert theIDF.get_idf("bilibalabulu") > 4.9, \
       "Término no localizado no manejado correctamente"
assert theIDF.get_idf("data") < theIDF.get_idf("radiology"), \
       "El idf de los términos más raros debe ser mayor que el de términos más comunes."
assert theIDF.get_idf("to") < theIDF.get_idf("design"), \
       "El idf de los términos más raros debe ser mayor que el de términos más comunes."
print("Tests de clase Idf() superados.")

Tests de clase Idf() superados.
